### deribit api example
Small python request example

In [ ]:
import json, pprint
import datetime, time
from ciso8601 import parse_datetime
import pandas as pd

In [ ]:
# make trades folder if it doesn't exist
import os
if not os.path.exists('trades'):
    os.makedirs('trades')

In [ ]:
# load secrets
secrets = json.load(open('./secrets.json'))
API_KEY = secrets['deribit']['key']
API_SECRET = secrets['deribit']['secret']

In [ ]:
# helpers
pp = pprint.PrettyPrinter(indent=2)
parseTimestamp = lambda x: datetime.datetime.fromtimestamp(x/1000.0)

In [ ]:
# instantiate deribit API
from DeribitAPI import DeribitClient
deribit = DeribitClient(api_key=API_KEY, api_secret=API_SECRET)

#### sample usage
Using some class functions with arbitrary params

In [ ]:
# EXAMPLE 1: get ETH option instruments and put in a dataframe and display first 5 results
instruments = deribit.get_instruments('ETH', 'option', False)
df_instruments = pd.DataFrame(instruments)
eth_options = df_instruments[df_instruments['instrument_name'].str.contains('ETH')]
eth_options.head()

In [ ]:
# EXAMPLE 2: Get last 10 ETH-28MAY21-2000-P trades
last_trades = deribit.get_last_trades_by_instrument(instrument='ETH-28MAY21-2000-P')
print('TRADES', len(last_trades['trades']))
pp.pprint(last_trades['trades'][0])

In [ ]:
# EXAMPLE 3: Get all trades for ETH-28MAY21-2000-P between 2021-4-24 and 2021-4-26
all_trades = deribit.get_all_trades_by_instrument(
    instrument='ETH-28MAY21-2000-P',
    start=int(datetime.datetime(2021, 4, 24).timestamp()*1e3),
    end=int(datetime.datetime(2021, 4, 26).timestamp()*1e3))

# output count of trades
print('TRADES', len(all_trades))

In [ ]:
# output parsed timestamps for trades
[parseTimestamp(x['timestamp']) for x in all_trades]

#### data export
Example implementation of data export functionality with parameters

In [ ]:
# conditions (parameters)
start_dt = datetime.datetime(2021, 4, 24) # from and including
end_dt = datetime.datetime(2021, 4, 25)   # up to but not including
currency_symbol = 'ETH'
instrument_filter = '30APR' # leave this as instrument_filter = None if you want to get all (could take a while)

# retrieve filtered instruments
active_options = pd.DataFrame(deribit.get_instruments(currency_symbol, 'option', False))
if instrument_filter:
    filtered_active_options = active_options[active_options['instrument_name'].str.contains(instrument_filter)]
else:
    filtered_active_options = active_options

# fetch and save trades within timeframe to respective csv
print('FETCHING TRADES FOR', len(filtered_active_options), 'INSTRUMENTS FOR', start_dt.strftime("%Y%m%d"))
for index, row in filtered_active_options.iterrows():
    
    # request all trades for instrument within timeframe
    instrument_trades = deribit.get_all_trades_by_instrument(
        instrument=row['instrument_name'],
        start=int(start_dt.timestamp()*1e3),
        end=int(end_dt.timestamp()*1e3))
    
    # parse to dataframe and save to csv
    trades_df = pd.DataFrame(instrument_trades)
    print(start_dt.strftime("%Y%m%d"), row['instrument_name'], 'TRADES:', len(trades_df))
    trades_df.to_csv(f'trades/{start_dt.strftime("%Y%m%d")}_{row["instrument_name"]}.csv', index=False)

#### playground
some python functionality to understand how to prepare parameters for the requests

In [ ]:
# deribit timestamp logic (conversion to/from python)
deribit_timestamp = 1619511419453
dt = datetime.datetime.fromtimestamp(deribit_timestamp / 1000.0)
reconverted = int(dt.timestamp()*1e3)
print(deribit_timestamp, dt, reconverted, dt.strftime('%Y%m%d'))
assert(deribit_timestamp == reconverted) # MUST NOT THROW ERROR